# Погодный бот, который строит карты

Этот код создаёт бота, который может строить карты максимальной температуры воздуха, средней влажности и максимального давления за последнюю неделю по любому из 85 субъектов РФ, который укажет пользователь. 

Климатические данные берутся с сайта "Погода и климат". 

In [ ]:
# Установка
!pip install beautifulsoup4 geopandas pyTelegramBotAPI mapclassify

In [3]:
#Импорт библиотек
import pandas as pd
import requests 
from bs4 import BeautifulSoup
import re
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np
import telebot
from google.colab import files
from time import sleep

In [ ]:
# Загрузка geojson файла, актуально только для colab
uploaded = files.upload()
#Обозначение переменных:
#Основная ссылка на сайт "Погода и климат"
URL_main = r'http://www.pogodaiklimat.ru/archive.php?id=ru'
#Заголовок, который желателен для получения ответа от интернет-страницы
HEADERS = {'user-agent' : 'user-agent', 'accept': '*/*'}
#Путь к файлу с административно-территориальным делением субъекта
path = r'Путь к файлу'
regions = gpd.read_file(path)
#Словарь, содержащий название колонок в наборе данных и расшифровку этих колонок 
title_dict = {'T_max' : 'Максимальная температура за последнюю неделю', 
              'f_mean' : 'Среднее значение влажности за последнюю неделю', 
              'P_max' : 'Максимальное давление за последнюю неделю'}



#Получение ответа от страницы 
def get_html (url):
    global HEADERS
    r = requests.get(url, headers=HEADERS)
    r.encoding = 'utf8'
    return r

#Получение ссылки на архив погоды для конкретного субъекта с общей страницы
def read_link (URL, region):
    r = get_html(URL)
    soup = BeautifulSoup(r.text, 'lxml')
    #По всей странице ищем ul c class='archive-list-link'
    links = soup.find_all('li', class_ = 'big-blue-billet__list_link')
    for link in links:
      if link.text == region:
        return 'http://www.pogodaiklimat.ru' + link.find('a')['href']
    else:
      return 'Проверьте свой ввод'

# Чтение ссылки на архив погоды для субъекта, получения словаря со ссылками на архивы для городов в этом субъекте
def read_sub_link (URL):
    city_links_dict = {}
    r = get_html(URL)
    soup = BeautifulSoup(r.text, 'lxml')
    #По всей странице ищем ul c class='archive-list-link'
    links = soup.find_all('li', class_ = 'big-blue-billet__list_link')
    for link in links:
      if 'id' in link.find('a')['href']:
        city_links_dict[link.text] = 'http://www.pogodaiklimat.ru' + link.find('a')['href']
      else:
        return city_links_dict

#Функция получения координат и названия пункта с интернет-страницы метеостанции
def get_coordinates (URL):
    st = ''
    r = get_html(URL)
    soup = BeautifulSoup(r.text, 'lxml')
    #По всей странице ищем div c class='archive-text'
    coordinates = soup.find('div', class_ = 'archive-text').text
    place = coordinates.split(' ')[2]
    lat = float(re.findall(r'\d+\.\d+' , coordinates)[0])
    lon = float(re.findall(r'\d+\.\d+' , coordinates)[1])
    return place, lat, lon 


#Функция, которая выбирает размер маркера в зависимости от величины показателя
def markers_size(GDF, col):
    size = GDF[col].to_numpy()
    s = [s for s in np.sort(size, axis=None)]
    s_2 = [i for i in range (len(s)*8, len(s)*16 + 8, 8)]
    s_3 = []
    for i in size: 
        for j in range(len(s)): 
            if i == s[j]:
                s_3.append(s_2[j]) 
                break
    return s_3

#Функция построения карты, добавления подписей и сохранения карты в формате png
def plot_labels_and_save(GDF, col_name, s, tit, region_name):
    fig, ax1 = plt.subplots(1, figsize = (15, 8))
    regions[regions['name'] == region_name].plot(ax = ax1, color='white', edgecolor='grey')
    GDF.plot(ax = ax1, column = col_name, cmap = 'cool', legend = True, s = s, scheme="quantiles")
    for x, y, label in zip(GDF.geometry.x, GDF.geometry.y, GDF['place']):
        ax1.annotate(label, xy=(x, y), xytext=(0, 8), textcoords="offset points").set_fontsize(10)
    ax1.set_title(f'{tit}, {region_name}').set_fontsize(12)
    plt.savefig(f'{tit}, {region_name}' + '.png', dpi=300)

def main():
    global URL_main, HEADERS, path, regions, title_dict

    #Подключение к телеграм-боту 
    bot = telebot.TeleBot('токен')

    # Ответ бота на его запуск
    @bot.message_handler(commands=["start"])
    def start(message):
        bot.send_message(message.chat.id, "Доброго времени суток!\nЯ могу построить карту погоды по любому из субъектов РФ\nВведите название субъекта:")

    #Ответ бота на любое сообщение пользователя (отправка полученных карт)
    @bot.message_handler(content_types=["text"])
    def handle_text(message):
      # Записываем название региона
        region_name = message.text
        
        #Создание пустого набора данных
        DF = pd.DataFrame(data = None, columns = ['place', 'lat', 'lon', 'T_max', 'f_mean', 'P_max'])
        
        # Получаем ссылки на cубъект и все города нужного субъекта
        sub_link = read_link(URL_main, region_name)
        if sub_link == 'Проверьте свой ввод':
             bot.send_message(message.chat.id, sub_link)
             return
        else:
            bot.send_message(message.chat.id, 'Процесс идёт, ожидайте...')

        sleep(5)
        city_links = read_sub_link(sub_link)

        # Получаем значения максимальной температуры, средней влажности и максимального давления 
        # за последние 7 дней для каждой метеостанции, добавляем полученные значения в набор данных 
        # Проходимся по ссылкам на все города нужного субъекта
        ind = 0
        for city in city_links:
            #Получаем таблицы со значениями показателей погоды для каждой из метеостанции в словаре
            sleep(1)
            df_city = pd.read_html(city_links[city])[1]
            df_city.drop(axis = 0, labels = 0, inplace = True)
            df_city.columns = ['Wind_dir', r'Wind_s_m/s', 'Vis_m', 'Ph', 'Cloudy', 'T_C', 'Td_C',
                      'f_%', 'Te_C', 'Tes_c', 'Comf', 'P_gPa', 'Po_gPa', 'Tmin_C', 'Tmax_c', 
                        'R_mm', 'R24_mm', 'S_sm']
            #Меняем формат необходимых столбцов 
            for col in ['T_C','f_%','P_gPa']:
                df_city[col] = pd.to_numeric(df_city[col])

            #Получаем название населённого пункта и координаты, соответствующие метеостанции
            place, lat, lon = get_coordinates(city_links[city])

            #Добавляем всю полученную информацию в набор данных 
            DF.loc[ind] = [place, lat, lon, df_city['T_C'].max(), df_city['f_%'].mean(), df_city['P_gPa'].max()]
            ind += 1
          
        #Переводим набор данных в географический набор данных, задаём систему координат
        GDF = gpd.GeoDataFrame(DF, geometry=gpd.points_from_xy(DF.lon, DF.lat))
        GDF.crs = "EPSG:4326"

        #Построение карт
        for titl in title_dict:
            s = markers_size(GDF, titl)
            plot_labels_and_save(GDF, titl, s, title_dict[titl], region_name)
          
        for titl2 in title_dict:
            bot.send_message(message.chat.id, (f'Вот {title_dict[titl2]}, {region_name}'))
            sti = open(f'{title_dict[titl2]}, {region_name}' + '.png', 'rb')
            bot.send_sticker(message.chat.id, sti)
    
    bot.polling(none_stop=True, interval=0)
    

In [ ]:
# Запуск
if __name__ == '__main__':
  main()